# Eat Safe, Love

## Notebook Set Up

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd


In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
# List collections in the database
collection_names = db.list_collection_names()

# Print the collection names
for name in collection_names:
    print(name)


establishments


In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [6]:
# Find the establishments with a hygiene score of 20
query = {"RatingValue": 20}
# Use count_documents to display the number of documents in the result
document_count = establishments.count_documents(query)
print("Number of documents:", document_count)

# Display the first document in the results using pprint
result_document = establishments.find_one(query)
pprint(result_document)

Number of documents: 0
None


In [7]:
# Convert the result to a Pandas DataFrame
cursor = establishments.find(query)
df = pd.DataFrame(list(cursor))
# Display the number of rows in the DataFrame
print("Number of rows in DataFrame:", len(df))

# Display the first 10 rows of the DataFrame
print(df.head(10))


Number of rows in DataFrame: 0
Empty DataFrame
Columns: []
Index: []


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [8]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {"LocalAuthorityName": {"$regex": "London", "$options": "i"}, "RatingValue": {"$gte": 4}}

# Use count_documents to display the number of documents in the result
document_count = establishments.count_documents(query)
print("Number of documents:", document_count)

# Display the first document in the results using pprint
result_document = establishments.find_one(query)
pprint(result_document)

Number of documents: 33
{'AddressLine1': 'Oak Apple Farm Building 103 Sheernes Docks',
 'AddressLine2': 'Sheppy Kent',
 'AddressLine3': '',
 'AddressLine4': '',
 'BusinessName': "Charlie's",
 'BusinessType': 'Other catering premises',
 'BusinessTypeID': 7841,
 'ChangesByServerID': 0,
 'Distance': 4627.439467780196,
 'FHRSID': 621707,
 'LocalAuthorityBusinessID': 'PI/000025307',
 'LocalAuthorityCode': '508',
 'LocalAuthorityEmailAddress': 'publicprotection@cityoflondon.gov.uk',
 'LocalAuthorityName': 'City of London Corporation',
 'LocalAuthorityWebSite': 'http://www.cityoflondon.gov.uk/Corporation/homepage.htm',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'ME12',
 'RatingDate': '2021-10-18T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': 4,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6574a440b5fe0454d16d3096'),
 'geocode': {'latitude': {'$toDouble': '$geocode.latitude'},
             'longitude': {'$toDouble': '$geocode.longitude'}},
 'links': [{'hr

In [9]:
# Convert the result to a Pandas DataFrame
cursor = establishments.find(query)
df = pd.DataFrame(list(cursor))
# Display the number of rows in the DataFrame
print("Number of rows in DataFrame:", len(df))

# Display the first 10 rows of the DataFrame
print(df.head(10))


Number of rows in DataFrame: 33
                        _id   FHRSID  ChangesByServerID  \
0  6574a440b5fe0454d16d3096   621707                  0   
1  6574a440b5fe0454d16d33be  1130836                  0   
2  6574a441b5fe0454d16d3f0b   293783                  0   
3  6574a441b5fe0454d16d4d0d   294474                  0   
4  6574a441b5fe0454d16d4d16  1315095                  0   
5  6574a441b5fe0454d16d5419   294900                  0   
6  6574a442b5fe0454d16d752a   293756                  0   
7  6574a442b5fe0454d16d78e9   878523                  0   
8  6574a442b5fe0454d16d7900   294606                  0   
9  6574a442b5fe0454d16d7903   293772                  0   

  LocalAuthorityBusinessID                          BusinessName  \
0             PI/000025307                             Charlie's   
1             PI/000034075               Mv City Cruises Erasmus   
2             PI/000002614             Benfleet Motor Yacht Club   
3             PI/000014647              Tilbur

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [10]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score
degree_search = 0.01
latitude = 145.1314  
longitude = 37.7877  

query = {
    "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search},
    "RatingValue": 5
}

sort = [("Scores.Hygiene", 1)]  

# Print the results
cursor = establishments.find(query).sort(sort).limit(5)
for document in cursor:
    pprint(document)


In [11]:
# Convert result to Pandas DataFrame
df = pd.DataFrame(list(cursor))

# Display the DataFrame
print(df)

Empty DataFrame
Columns: []
Index: []


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [12]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
client = MongoClient("mongodb://localhost:27017/")
database = client.uk_food
establishments = database["establishments"]
# Create an aggregation pipeline
pipeline = [
    {"$match": {"RatingValue": 0}},  # Match establishments with a hygiene score of 0
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},  # Group by Local Authority and count
    {"$sort": {"count": -1}},  # Sort by count in descending order
    {"$limit": 10}  # Limit to the top 10 results
]

# Execute the aggregation pipeline
result_cursor = establishments.aggregate(pipeline)

# Cast the result as a list
result_list = list(result_cursor)

# Print the number of documents in the result
print("Number of establishments with hygiene score 0 in each Local Authority:")
pprint(result_list[:10])


Number of establishments with hygiene score 0 in each Local Authority:
[{'_id': 'Newham', 'count': 13},
 {'_id': 'Waltham Forest', 'count': 9},
 {'_id': 'Greenwich', 'count': 7},
 {'_id': 'Uttlesford', 'count': 6},
 {'_id': 'Barking and Dagenham', 'count': 4},
 {'_id': 'Redbridge', 'count': 4},
 {'_id': 'Southend-On-Sea', 'count': 3},
 {'_id': 'Tendring', 'count': 3},
 {'_id': 'Rochford', 'count': 2},
 {'_id': 'Havering', 'count': 1}]


In [13]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(result_list)
# Display the number of rows in the DataFrame
print("\nNumber of rows in the DataFrame:", len(df))


# Display the first 10 rows of the DataFrame
print("\nTop 10 Local Authority areas with hygiene score 0:")
print(df.head(10))


Number of rows in the DataFrame: 10

Top 10 Local Authority areas with hygiene score 0:
                    _id  count
0                Newham     13
1        Waltham Forest      9
2             Greenwich      7
3            Uttlesford      6
4  Barking and Dagenham      4
5             Redbridge      4
6       Southend-On-Sea      3
7              Tendring      3
8              Rochford      2
9              Havering      1
